In [1]:
%autoreload 2

In [2]:
from argparse import Namespace
from collections import defaultdict
import copy
from datetime import datetime
import difflib
import duckdb
import gzip
import itertools
import os
import pickle
import sys
import typing

from IPython.display import display, Markdown, HTML  # type: ignore
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import tatsu
import tqdm.notebook as tqdmn


sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../reward-machine'))

import compile_predicate_statistics
import compile_predicate_statistics_split_args
from config import SPECIFIC_NAMED_OBJECTS_BY_ROOM

2023-08-07 14:45:42 - ast_utils - DEBUG    - Using cache folder: /tmp/game_generation_cache


In [3]:
cache_dir = compile_predicate_statistics.get_project_dir() + '/reward-machine/caches'

# regular_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics.pkl'))
split_args_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics_028b3733.pkl.gz'))
# split_args_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics_4d5dd602.pkl.gz'))

# stats = compile_predicate_statistics.CommonSensePredicateStatistics(cache_dir)
split_args_stats = compile_predicate_statistics_split_args.CommonSensePredicateStatisticsSplitArgs(
    cache_dir, compile_predicate_statistics_split_args.CURRENT_TEST_TRACE_NAMES, overwrite=False)

2023-08-07 14:45:47 - compile_predicate_statistics_split_args - INFO     - Loaded data with shape (416740, 8) from /Users/guydavidson/projects/game-generation-modeling/reward-machine/caches/predicate_statistics_4d5dd602.pkl.gz


After filtering out same_type, data has shape (141198, 8)


In [9]:
numbers = list(range(10))

bits_df = pl.DataFrame(dict(nums=numbers, bits_1=[[0, 1] for n in numbers], bits_2=[[1, 0] for n in numbers]), 
                       schema_overrides=dict(bits_1=pl.List(pl.UInt8), bits_2=pl.List(pl.UInt8)))

bits_df

nums,bits_1,bits_2
i64,list[u8],list[u8]
0,"[0, 1]","[1, 0]"
1,"[0, 1]","[1, 0]"
2,"[0, 1]","[1, 0]"
3,"[0, 1]","[1, 0]"
4,"[0, 1]","[1, 0]"
5,"[0, 1]","[1, 0]"
6,"[0, 1]","[1, 0]"
7,"[0, 1]","[1, 0]"
8,"[0, 1]","[1, 0]"


In [11]:
bits_df.select(pl.col('bits_1') + pl.col('bits_2'))

thread '<unnamed>' panicked at '`add` operation not supported for dtype `list[u8]`', /Users/runner/work/polars/polars/polars/polars-core/src/series/series_trait.rs:163:13


PanicException: `add` operation not supported for dtype `list[u8]`

In [ ]:
bits_df.select(pl.col('bits_1') & pl.col('bits_2'))

In [ ]:
all_specific_names = set()
for room, type_to_ids in SPECIFIC_NAMED_OBJECTS_BY_ROOM.items():
    all_specific_names.update(type_to_ids.keys())

specific_name_rows = split_args_df[(split_args_df.arg_1_type.isin(all_specific_names) | split_args_df.arg_2_type.isin(all_specific_names))].shape[0]
total_rows = split_args_df.shape[0]
print(f'{specific_name_rows} / {total_rows} ({specific_name_rows / total_rows * 100:.2f}%) rows have a specific name')

In [ ]:
same_type_rows = split_args_df[(split_args_df.predicate == 'same_type')].shape[0]
print(f'{same_type_rows} / {total_rows} ({same_type_rows / total_rows * 100:.2f}%) rows are for same_type')

In [ ]:
for i, row in split_args_df.iterrows():
    print(i, row.to_dict())
    break

In [ ]:
split_args_df[(split_args_df.predicate == 'adjacent') & (split_args_df.arg_1_type == 'agent') & (split_args_df.arg_2_type == 'green_golfball')]

In [ ]:
split_args_df[(split_args_df.predicate == 'same_type')]

In [ ]:
split_args_df[(split_args_df.predicate == 'in') & (split_args_df.arg_1_type == 'building')].arg_2_type.value_counts()

In [ ]:
door_df = split_args_df[(split_args_df.arg_1_type == 'door') | (split_args_df.arg_1_type == 'door')]
print(door_df.predicate.value_counts())
print()
for predicate in door_df.predicate.unique():
    pred_df = door_df[door_df.predicate == predicate]
    print(f'For predicate {predicate}:')
    print(pred_df.arg_2_type.value_counts().iloc[:10])
    print()

In [ ]:
for tid in door_df[door_df.predicate == 'in_motion'].trace_id:
    print(tid)

In [ ]:
split_args_df[split_args_df.predicate == 'in_motion'].arg_1_type.value_counts()

In [ ]:
stats._invert_intervals(regular_df[(regular_df.trace_id == trace_id) & (regular_df.predicate == 'in_motion') & (regular_df.arg_ids == (ball_id,))].intervals.values[0],
                        stats.trace_lengths[trace_id])

In [ ]:
split_args_df[(split_args_df.trace_id == trace_id) & (split_args_df.predicate == 'in') & (split_args_df.arg_1_id == bin_id) & (split_args_df.arg_2_id == ball_id)]

In [ ]:
ball_not_in_motion_int = split_args_stats._invert_intervals(
    split_args_df[(split_args_df.trace_id == trace_id) & (split_args_df.predicate == 'in_motion') & (split_args_df.arg_1_id == ball_id)].intervals.values[0],
    split_args_stats.trace_lengths_and_domains[trace_id][0],
)

ball_not_in_motion_int

In [ ]:
ball_in_bin_int = split_args_df[(split_args_df.trace_id == trace_id) & (split_args_df.predicate == 'in') & (split_args_df.arg_1_id == bin_id) & (split_args_df.arg_2_id == ball_id)].intervals.values[0]

split_args_stats._intersect_intervals(ball_in_bin_int, ball_not_in_motion_int)

In [ ]:
single_trace_split_args_df = split_args_df[split_args_df['trace_id'] == trace_id]

single_trace_split_args_stats = compile_predicate_statistics_split_args.CommonSensePredicateStatisticsSplitArgs(cache_dir)
single_trace_split_args_stats.data = pl.from_pandas(single_trace_split_args_df)
single_trace_split_args_stats.data.shape

In [ ]:
DEFAULT_GRAMMAR_PATH = "../dsl/dsl.ebnf"
grammar = open(DEFAULT_GRAMMAR_PATH).read()
grammar_parser = typing.cast(tatsu.grammars.Grammar, tatsu.compile(grammar))

game = open(compile_predicate_statistics.get_project_dir() + '/reward-machine/games/ball_to_bin_from_bed.txt').read()
game_ast = grammar_parser.parse(game) 

test_pred_1 = game_ast[4][1]['preferences'][0]['definition']['forall_pref']['preferences']['pref_body']['body']['exists_args']['then_funcs'][1]['seq_func']['hold_pred']

# should be: (and (not (in_motion ?b)) (in ?h ?b)))
test_pred_2 = game_ast[4][1]['preferences'][0]['definition']['forall_pref']['preferences']['pref_body']['body']['exists_args']['then_funcs'][2]['seq_func']['once_pred']

In [ ]:
test_mapping = {"?b": ["ball"], "?h": ["hexagonal_bin"]}
test_out = single_trace_split_args_stats.filter(test_pred_2, test_mapping)
test_out